In [10]:
import pandas as pd
from selenium import webdriver
from time import sleep
from IPython.display import HTML, display
from tqdm import tqdm, trange

In [11]:
driver = webdriver.Chrome(r'/Users/aliaksandr.lashkov/Applications/chromedriver')
# driver.switch_to_frame(0)

In [9]:
driver.get('https://login.aliexpress.com')
# username = driver.find_element_by_name("fm-login-id")
# username.send_keys('pop4irka@gmail.com')
# sleep(1)
# password = driver.find_element_by_name("fm-login-password")
# password.send_keys('Dtctkfz45pyflyj')
# sleep(1)
# submit = driver.find_element_by_css_selector('button.fm-button')
# submit.click()
# sleep(10)

In [ ]:
orders_page = 'https://trade.aliexpress.com/orderList.htm'
driver.get(orders_page)
# sleep(5)

In [13]:
paginator = driver.find_element_by_class_name('ui-pagination-navi.util-left')
max_page = max(list(map(int, paginator.text.split('\n')[1:-1])))
df = pd.DataFrame(columns=['img', 'desc', 'tracking_number', 'status', 'tracking_steps'])
for page_num in trange(1, max_page + 1, desc=f'working on page'):
    driver.get(orders_page)
    sleep(2)
    if page_num != 1:
        page_btn = driver.find_element_by_xpath(f"//a[@class='ui-goto-page' and text()='{page_num}']")
        page_btn.click()
#         sleep(5)

    page_orders = driver.find_elements_by_class_name('order-item-wraper')
    order_ids = []
    for order in tqdm(page_orders, desc=f'working on page orders'):
        order_id = order.find_element_by_class_name('info-body').text
        order_ids.append(order_id)
        img_url = order.find_element_by_tag_name('img').get_attribute('src')
        df.loc[order_id, 'img'] = img_url
        df.loc[order_id, 'desc'] = order.find_element_by_class_name('baobei-name').get_attribute('title')
        df.loc[order_id, 'status'] = order.find_element_by_class_name('f-left').text

    for order_id in order_ids:
        driver.get(f'https://track.aliexpress.com/logisticsdetail.htm?tradeId={order_id}')
        sleep(1)
        tracking_numbers = driver.find_elements_by_class_name('tracking-no')
        if tracking_numbers:
            df.loc[order_id, 'tracking_number'] = tracking_numbers[0].text
        ship_steps = driver.find_elements_by_css_selector('.step')
        if ship_steps:
            tracking_steps = []
            for step in ship_steps:
                time = step.find_element_by_css_selector('.step-time').text.replace('\n', ' ')
                step_content = step.find_element_by_css_selector('.step-content').text.replace('\n', ' ')
                tracking_steps.append(f'{time} {step_content}')
            df.loc[order_id, 'tracking_steps'] = '\n'.join(tracking_steps)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ui-pagination-navi.util-left"}
  (Session info: chrome=95.0.4638.69)


In [ ]:
pd.options.display.max_colwidth = -1
df_in_progress = df.loc[~df['status'].isin(['Finished', 'Closed'])]
df_in_progress = df_in_progress.drop('desc', axis=1)
fmts = {'img': lambda img_url: f'<img src="{img_url}" width="70">',
        'tracking_steps': lambda x: x.replace('\n', '<br>')}
display(HTML(df_in_progress.to_html(escape=False, formatters=fmts)))

In [ ]:
driver.close()